### 01-AI-Nutritionist-Demo

In [1]:
import os
import chromadb
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, ServiceContext, get_response_synthesizer, Settings, PromptTemplate
#from llama_index.readers.file import PDFReader, CSVReader
from IPython.display import display, Markdown
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.embeddings.openai import OpenAIEmbedding

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LLAMA_INDEX_API_KEY = os.getenv("LLAMA_INDEX_API_KEY")

In [5]:
import nest_asyncio

nest_asyncio.apply()

In [6]:
from llama_parse import LlamaParse
    
# Initialize the LlamaParse object with parameters
parser = LlamaParse(
    result_type="markdown",
    api_key=LLAMA_INDEX_API_KEY,
    verbose=True,
    language="en",
    num_workers=8
)

In [7]:
# use SimpleDirectoryReader to parse our file
file_extractor = {".pdf": parser}

reader = SimpleDirectoryReader(input_dir =r"D:\Generative-AI\03-Generative-AI-Frameworks\data\aiNutritionist\Clinical_Reports", file_extractor=file_extractor)
clinicalReports = reader.load_data()

reader = SimpleDirectoryReader(input_dir =r"D:\Generative-AI\03-Generative-AI-Frameworks\data\aiNutritionist\Diet_Plan", file_extractor=file_extractor)
dietPlan = reader.load_data()

reader = SimpleDirectoryReader(input_dir =r"D:\Generative-AI\03-Generative-AI-Frameworks\data\aiNutritionist\ICMR_Guidelines", file_extractor=file_extractor)
icmrGuidelines = reader.load_data()




Started parsing the file under job_id d93b0724-0bec-43a2-b436-a16a4dd4ff7d
Started parsing the file under job_id eb7b0279-4a28-4e96-a0df-72612f990099
Started parsing the file under job_id 4a2df5e5-a115-4867-a432-765d7e6252d1
Started parsing the file under job_id 94b56ae0-be05-4509-8014-a576788df786
Started parsing the file under job_id 75abb309-125d-43d2-a69a-d8c5c006a575
Started parsing the file under job_id 7ad9691b-0ce4-4d5d-a6eb-89a7a7e1a511
.

In [9]:
documents= clinicalReports + dietPlan + icmrGuidelines

In [11]:
for i in range(len(documents)):  # Iterate through documents
    chunk_size = len(documents[i].text[0:50000])  
    print(f"Chunk size (characters) for document {i}: {chunk_size}")

Chunk size (characters) for document 0: 22
Chunk size (characters) for document 1: 22
Chunk size (characters) for document 2: 109
Chunk size (characters) for document 3: 49
Chunk size (characters) for document 4: 7289
Chunk size (characters) for document 5: 50000


In [12]:
from llama_index.embeddings.openai import OpenAIEmbedding
# Load global settings
#Settings.api_key = OPENAI_API_KEY

# Load in a specific embedding model
embed_model = OpenAIEmbedding(model="text-embedding-3-large", chunk_size=1024, chunk_overlap=200, embed_batch_size=32,api_key=OPENAI_API_KEY)

# Load in a specific language model
llm = OpenAI(model="gpt-4o", api_key=OPENAI_API_KEY,temperature=0.7, max_tokens=3000, top_p=1, logprobs=True, echo=True, stream=True)


# Load global settings
Settings.embed_model = embed_model
Settings.llm = llm

In [13]:
# Create Custom Prompt Template

template = (
    "You are NutriGuide, an AI assistant specializing in personalized nutrition advice. You provide diet plans and nutritional guidance based on ICMR guidelines and individual health reports. Start each response with 'NutriGuide:'.\n"
    "**Context Information**\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "**User Question:** {query_str}\n"
)


context_str = (
    "User Health Report Summary:\n"
    "- Condition: High blood pressure\n"
    "- Age: 34\n"
    "- Gender: Female\n"
    "- Weight: 74 kg\n"
    "- Height: 160 cm\n"
    "- Lifestyle: Sedentary\n"
    "- Specific dietary restrictions: None\n"
)


qa_template = PromptTemplate(template)


In [14]:
# initialize client
db = chromadb.PersistentClient(path="./chroma_db")

# get collection
chroma_collection = db.get_or_create_collection("quickstart")

# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model, llm=llm
)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

In [15]:
query_engine = index.as_query_engine(text_qa_template=qa_template)


In [18]:
response = query_engine.query("AS you know that user has high blood pressure and colestarol what food should he avoid?")


In [19]:
display(Markdown(f"{response}"))

NutriGuide: For individuals with high blood pressure and cholesterol, it is crucial to manage their diet to help control these conditions. Based on the ICMR guidelines, here are the foods that should be avoided:

### Foods to Avoid for High Blood Pressure:
1. **High-Salt Foods**:
   - Processed and packaged foods like chips, sauces, biscuits, bakery products.
   - Home-prepared foods like savory snacks, namkeen, papads, and pickles.
   - Beverages and foods where salt is added by the manufacturer, cook, or consumer.
   - Preserved meats/vegetables and ready-to-eat foods.
   - Additives like baking soda, baking powder, and monosodium glutamate in processed foods.
   
2. **Sodium-Rich Foods**:
   - Excessive use of table salt.
   - Foods with added salt during cooking or processing.
   - Canned foods and salted dry fish.
   - Salted butter, cheese, and salted nuts/dry fruits.

### Foods to Avoid for High Cholesterol:
1. **High Saturated Fat Foods**:
   - Red meats like beef, mutton, pork.
   - High-fat dairy products such as full-cream milk, cheese, and butter.
   - Foods prepared using ghee, butter, palm oil, or coconut oil.
   - Deep-fried snacks and sweets.
   
2. **Trans Fats**:
   - Fast foods, baked goods like cakes, pastries, and cookies.
   - Margarine and shortening.
   - Many commercially fried foods and snacks.

### General Recommendations:
- **Limit Added Sugars**: Avoid foods and beverages with added sugars, such as sugary drinks, candies, and desserts. Added sugars should not exceed 5% of total energy intake per day or 25g/day.
- **Increase Potassium Intake**: Include potassium-rich foods like fresh vegetables, fruits, beans, lentils, bananas, dry fruits, and coconut water to balance sodium intake.
- **Whole Foods**: Focus on whole, minimally processed foods which are naturally low in sodium and unhealthy fats.

### Lifestyle Tips:
- **Read Labels**: Always check food labels for sodium, saturated fats, and trans fats content.
- **Cooking Methods**: Opt for grilling, steaming, or baking instead of frying.
- **Seasoning Alternatives**: Use herbs and spices instead of salt to flavor foods.
- **Hydration**: Drink plenty of water to support overall health.

By avoiding these foods and following these guidelines, individuals with high blood pressure and cholesterol can better manage their conditions and improve their health.